In [2]:
%load_ext autoreload
%notebook autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import numpy as np
import geopandas as gpd
from scipy.spatial import KDTree
from tqdm.autonotebook import tqdm
from scipy.ndimage import gaussian_filter
from fuzzywuzzy import process
import contextily as cx

from IndexUtils import * 
from TileUtils import *
from featurematching import *

# TILED INFERENCE
import sahi
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction, predict
from scipy.spatial.distance import cdist

from scipy.fftpack import fft2, ifft2, fftshift
from scipy.ndimage import center_of_mass

from sklearn.cluster import DBSCAN

from skimage.morphology import skeletonize

from scipy.interpolate import Rbf

C:\Users\fhacesga\AppData\Local\Temp\ipykernel_22360\2864581687.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


WARNING  Python>=3.10 is required, but Python==3.8.18 is currently installed 


In [4]:
corners = gpd.read_file(r"C:\Users\fhacesga\OneDrive - University Of Houston\AAA_RECTDNN\data\AAA_ReferenceDatasets\OSM_Roads\TexasRoads_filtered_strict_3857_nondupe_smaller_harris.gpkg")
roads_gdf = gpd.read_file(r"C:\Users\fhacesga\Downloads\TexasRoads_filtered_strict_3857.gpkg", engine='pyogrio', use_arrow=True)

In [5]:
with open("SmartDictV2.p", 'rb') as f:
    tile_coords = pickle.load(f)

In [6]:
with open(r"D:\RECTDNN\processing\2024-10-11_15-08-46\street_corners\exp7\pickles\48201C0660L.pickle", 'rb') as f:
    x = pickle.load(f)    
    corner_bbox = [a.bbox.to_xyxy() for a in x if a.score.value > 0.001]
    corner_arry = np.array([calcCenter(a) for a in corner_bbox ]) * 2
    print(f"{len(corner_bbox)} / {len(x)}")

1331 / 1331


In [7]:
# r"D:\RECTDNN\processing\2024-10-11_15-08-46\referencing\48201C0930J.tif"
im_file    = r"D:\RECTDNN\processing\2024-10-11_15-08-46\referencing\48201C0660L.tif"
him_file   = r"D:\RECTDNN\processing\2024-10-11_15-08-46\half_size\48201C0660L.tif"

curr_tn = os.path.basename(im_file.split(".")[0])[:-1]
curr_tf = os.path.basename(im_file.split(".")[0])[-1]

ra = rio.open(im_file)
bounds  = ra.bounds
image   = ra.read(1)

bounds, RLNN = findBounds(im_file, None)
bbox = bounds[0].boxes.xyxy.numpy().astype(np.int32)[0].flatten()

real_x, real_y = bbox_to_coords_realworld(tile_coords[curr_tn][curr_tf]["coords"][0])
pic_x, pic_y   = bbox_to_coords_raster(bbox)

affine = affineTransformation(pic_x, pic_y, real_x, real_y)

# corner_arry = streetcorners[os.path.basename(tile).split(".")[0] + ".pickle"] * 2# streetresults_struct[tile_coords[out_name]['input_file']]
corner_arry = np.int32(corner_arry)                 # CONVERT TO INT32
corner_arry = cleanCenterBBOX(corner_arry, bbox)

# USE RASTER TRANSFORM TO GET REAL-WORLD ESTIMATES OF STREET CORNERS
im_corner_gdf = pointsToGeodataFrame(ra, corner_arry[:, 1], corner_arry[:, 0])

bounds_poly = enlarged_bounds(ra, n=2)

descriptor_kwds = {
    "angle_step"        : 5,
    "azimuth_radius"    : 500
}

corners, _ = calcDescriptors(corners, **descriptor_kwds)
corners_curr = gpd.clip(corners, bounds_poly)

  0%|          | 0/127256 [00:00<?, ?it/s]

In [8]:
roads_clipped = gpd.clip(roads_gdf, bounds_poly)    # CLIP
streets = getRoadPoints(roads_clipped, 0.5)         # ROAD POINTS ALONG STREETS, 0.5 IN M

model = TPNN(num_classes=2, finalpadding=1, inputsize=2)
checkpoint = torch.load(r'C:\Users\fhacesga\OneDrive - University Of Houston\AAA_RECTDNN\data/FANN/checkpoint_101123.pth')
model.load_state_dict(checkpoint['model_state_dict'])
model = model.to('cuda')

roads, _    = split_and_run_cnn(image, model, tilesize=1024, num_dim=3, edges=10, dims_rep=[0], n_pyramids=2, device="cuda", verbose=True)
roads_thin  = roads[:, :, 1] > 0.95
roads_thin  = cleanImageBBOX(roads_thin, bbox)
roads_thin  = skeletonize(roads_thin)    # USE SKELETONIZATION ALGORITHM TO THIN ROADS
y, x        = np.where(roads_thin[:, :])            # GET X, Y COORDINATES OF IDENTIFIED ROAD POINTS
street_det  = pointsToGeodataFrame(ra, y, x)        # CONVERT X, Y COORDINATES TO GDF AND REPROEJCT TO CURRENT ESTIMATE OF RASTER POSITION

  0%|          | 0/11 [00:00<?, ?it/s]

In [9]:
compoundedAdjustment = np.eye(3)
flip=np.array([
    [1, 0, 0],
    [0, -1, 0],
    [0, 0, 1]
    ]) 


r_initial = 500

prev_transform = getMatrixFromAffine(ra.transform)

for i in range(20):

    r_current = r_initial * 0.9 ** i

    if i == 0:
        matching_results, im_corner_gdf, corners_curr = matching(im_corner_gdf, corners_curr, ratio_threshold=0.95, **descriptor_kwds)
        matrix, distances = iterativeAdjustFromMatching(matching_results, corners_curr=corners_curr, im_corner_gdf=im_corner_gdf, plot=False)
        #fromPoints, toPoints = iterativeAdjustFromMatching(matching_results, corners_curr=corners_curr, im_corner_gdf=im_corner_gdf, plot=True)
    else:
        matching_results, _, _ = matching_distances(im_corner_gdf_adj, corners_curr, match_radius=r_current, **descriptor_kwds)
        matrix, distances = iterativeAdjustFromMatching(matching_results, plot=False)

    compoundedAdjustment = np.linalg.inv(matrix) @ compoundedAdjustment 

    # current_adjustment = compoundedAdjustment @ getMatrixFromAffine(ra.transform).T
    # current_adjustment = getMatrixFromAffine(ra.transform) @ compoundedAdjustment

    rev_adj = compoundedAdjustment @ flip
    current_adjustment = getMatrixFromAffine(ra.transform) @ flip @ rev_adj

    reprojectedcorners = np.hstack((corner_arry, np.ones((corner_arry.shape[0], 1)))) @ current_adjustment.T

    if False:
        plt.figure()
        plt.scatter(reprojectedcorners[:, 0], reprojectedcorners[:, 1])
        test = np.hstack((corner_arry, np.ones((corner_arry.shape[0], 1)))) @  getMatrixFromAffine(ra.transform).T
        plt.scatter(test[:, 0], test[:, 1])
        plt.show()

    im_corner_gdf_adj = im_corner_gdf.copy()
    im_corner_gdf_adj['geometry'] = gpd.points_from_xy(reprojectedcorners[:,0], reprojectedcorners[:, 1])
    
    # register_transform1 = matrix @ register_transform
    print(np.nanpercentile(distances, [0, 25, 50, 75, 100]), len(matching_results))

print(current_adjustment - prev_transform)

with rio.open("test_registrationV4.tif", 'w',
                driver='GTiff', count=1, dtype=np.uint8,
                height=image.shape[0], width=image.shape[1],
                crs=f'EPSG:3857',
                transform=rio.Affine(*current_adjustment.flatten()[:6])) as dst:
                    dst.write(image, 1) 

  0%|          | 0/1280 [00:00<?, ?it/s]

  0%|          | 0/1280 [00:00<?, ?it/s]

[    0.99434      4.1996      6.8884      11.677      91.216] 555


  0%|          | 0/1280 [00:00<?, ?it/s]

  0%|          | 0/13178 [00:00<?, ?it/s]

...

  0%|          | 0/13178 [00:00<?, ?it/s]

[    0.85632      1.9686      3.0865       4.908      19.608] 190


  0%|          | 0/1280 [00:00<?, ?it/s]

  0%|          | 0/13178 [00:00<?, ?it/s]

[ 2.9118e-12  3.2749e-12   3.638e-12  3.6521e-12  3.6663e-12] 153
[[     0.1022    0.018474      2773.2]
 [   0.018662   -0.060042      918.65]
 [          0           0           0]]
